In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import falkon
import torch

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
data_path = "data/YearPredictionMSD.txt"
data_path = "data/MSD_small.csv"

In [4]:
### Load the Data ###
data = pd.read_csv(data_path,header=None)
X,y = data.drop(0, axis=1), data[0]

In [ ]:
#### Normalize ####
X = (X - X.mean()) / X.std()

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
X_train = torch.tensor(X_train.to_numpy())
y_train = torch.tensor(y_train.to_numpy())

X_test = torch.tensor(X_test.to_numpy())
y_test = torch.tensor(y_test.to_numpy())

print(f"{X_train.shape=},{y_train.shape=},{X_test.shape=},{y_test.shape=}")

X_train.shape=torch.Size([8, 90]),y_train.shape=torch.Size([8]),X_test.shape=torch.Size([3, 90]),y_test.shape=torch.Size([3])


In [8]:
sigma = 1
lam = 1e-2
m = 3

options = falkon.FalkonOptions(keops_active="no")
kernel = falkon.kernels.GaussianKernel(sigma=sigma, opt=options)
flk = falkon.Falkon(kernel=kernel, penalty=lam, M=m, options=options)
flk.fit(X,y)
flk.predict(X_test).reshape(-1, )

DeferredCudaCallError: CUDA call failed lazily at initialization with error: device >= 0 && device < num_gpus INTERNAL ASSERT FAILED at "../aten/src/ATen/cuda/CUDAContext.cpp":49, please report a bug to PyTorch. device=, num_gpus=

CUDA call was originally invoked at:

  File "/raid/home/fotis/.local/share/uv/python/cpython-3.9.22-linux-x86_64-gnu/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/raid/home/fotis/.local/share/uv/python/cpython-3.9.22-linux-x86_64-gnu/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/raid/home/fotis/.cache/uv/archive-v0/KDzk6SphcqjJcLzmzfKMj/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/raid/home/fotis/.cache/uv/archive-v0/KDzk6SphcqjJcLzmzfKMj/lib/python3.9/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/raid/home/fotis/.cache/uv/archiv